# Data Analysis of Data Analysts

Project 1

In [1]:
# Dependencies and Setup
# import os
import csv
import pandas as pd
# from pathlib import Path
# from matplotlib import pyplot as plt
import matplotlib.pyplot as plt
# from scipy.stats import sem
from scipy.stats import linregress
# import scipy.stats as st
import numpy as np
import requests
import json
from config import acct_id
from config import api_key 


In [2]:
city_state = {
    'City': ["Atlanta", "Austin", "San Francisco", "New York City", "San Diego", "Boston", "Portland",
            "Seattle", "Los Angeles", "Chicago", "Denver", "Dallas"],
    'State': ["GA", "TX", "CA", "NY", "CA", "MA", "OR", "WA", "CA", "IL", "CO", "TX"]}
    
dataframe_1 = pd.DataFrame(city_state)
dataframe_1[['# of Analysts - City', 'Median Salary - City']] = ""

dataframe_1

,City,State,# of Analysts - City,Median Salary - City
0,Atlanta,GA,,
1,Austin,TX,,
2,San Francisco,CA,,
3,New York City,NY,,
4,San Diego,CA,,
5,Boston,MA,,
6,Portland,OR,,
7,Seattle,WA,,
8,Los Angeles,CA,,
9,Chicago,IL,,


In [3]:
# Use the city API request but alter the city and state based on the dataframe
# This allows us to search national, state, and city data all from the same call

base_url = city_url = f"https://api.careeronestop.org/v1/occupation/{acct_id}/Data%20Scientists/"
params = "?training=false&interest=true&videos=false&tasks=false&dwas=false&wages=true&alternateOnetTitles=false&projectedEmployment=true&ooh=false&stateLMILinks=false&relatedOnetTitles=false&skills=false&knowledge=false&ability=false&trainingPrograms=false"
headers = {"Authorization": f"Bearer {api_key}"}

In [4]:
for index, row in dataframe_1.iterrows():
    # Get the city and state
    city = dataframe_1.loc[index, "City"]
    state = dataframe_1.loc[index, "State"]
    # Add the current City type to the parameters
    api_url =  f"{base_url}{city}%2C%20{state}{params}"
    
    response_1 = requests.get(api_url, headers = headers).json()
    results = response_1['OccupationDetail'][0]['Wages']['BLSAreaWagesList']
#     results_2 = response["OccupationDetail"][0]['Projections']['Projections'][0]['ProjectedEmployment']

    try:
#         should we use estimated employment OR projected employment?
        projected_jobs = response_1["OccupationDetail"][0]['Projections']['Projections'][0]['EstimatedEmployment']
        dataframe_1.loc[index, '# of Analysts - City'] = projected_jobs
        
#         help from tutor
        median_salary = [x['Median'] for x in results if x['RateType']=='Annual' ][0] 
        dataframe_1.loc[index, 'Median Salary - City'] = median_salary    

    except (KeyError, IndexError):
        dataframe_1.loc[index, 'Median Salary - City'] = "Didnt find anything"
        dataframe_1.loc[index, '# of Analysts - City'] = "Didnt find anything"

dataframe_1


,City,State,# of Analysts - City,Median Salary - City
0,Atlanta,GA,"2,250",104060
1,Austin,TX,"4,110",105350
2,San Francisco,CA,"18,400",137680
3,New York City,NY,"3,500",130070
4,San Diego,CA,"18,400",123570
5,Boston,MA,"2,880",106670
6,Portland,OR,760,108130
7,Seattle,WA,"2,700",140560
8,Los Angeles,CA,"18,400",103430
9,Chicago,IL,"3,370",104370


In [ ]:
#merge the 2 dataframes and use that for the heat map